Mediapipe contains a set of pre-trained models for various tasks, including face detection, hand tracking, and pose estimation. These models are designed to work efficiently on mobile devices and can be used in real-time applications.
in this notebook, we will explore how to use mediapipe for face detection and hand tracking. We will also discuss how to visualize the results using OpenCV.

**Note:** we are using the latest version of mediapipe (0.10.1) that means we have to import the model to create a task instead of using the old interface that involves using the `mediapipe.solutions` module. The new interface is more flexible and allows for easier integration with other libraries and frameworks.
# Import necessary libraries

In [4]:
import cv2
import mediapipe as mp

import os

from mediapipe.tasks import python
from mediapipe.tasks.python import vision

Model link https://ai.google.dev/edge/mediapipe/solutions/vision/hand_landmarker/index#models

In [5]:
# Load variables from .env file for local development or get it from the environment variables in production
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass  # Safe fallback if it's not available in prod

base_path = os.getenv("LSM_BASE")
if not base_path:
    raise ValueError("❌ Environment variable 'LSM_BASE' is not set!")

model_path = os.path.join(base_path, "models", "hand_landmarker.task")
image_path = os.path.join(base_path, "data", "images", "image-003.png")


In [6]:
# Setup MediaPipe classes
BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

# Create the landmarker instance
options = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.IMAGE,
    num_hands=2
)

with HandLandmarker.create_from_options(options) as hand_landmarker:
    # Load and process the image
    mp_image = mp.Image.create_from_file(image_path)
    result = hand_landmarker.detect(mp_image)

    # Load image with OpenCV for visualization
    image_bgr = cv2.imread(image_path)

    # Draw landmarks if found
    if result.hand_landmarks:
        for hand in result.hand_landmarks:
            for landmark in hand:
                x = int(landmark.x * image_bgr.shape[1])
                y = int(landmark.y * image_bgr.shape[0])
                cv2.circle(image_bgr, (x, y), 4, (0, 255, 0), -1)

        print("✅ Hands detected")
    else:
        print("❌ No hands detected")

    # Show the image with landmarks
    cv2.imshow("MediaPipe Hand Landmarks", image_bgr)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


✅ Hands detected
